In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import  confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense 

In [2]:
!nvidia-smi

Wed Nov  2 14:46:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 510.73.08    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro GP100        On   | 00000000:17:00.0 Off |                    0 |
| 32%   48C    P0    33W / 235W |      4MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro GP100        On   | 00000000:65:00.0 Off |                    0 |
| 33%   

In [3]:
data_frame_train = pd.read_csv("../../csv_files_new_ppi/training_and_test_set/train_set_subset_with_embedding.csv")
data_frame_test = pd.read_csv("../../csv_files_new_ppi/training_and_test_set/test_set_subset_with_embedding.csv")


In [6]:
train_features = data_frame_train.copy()
train_labels = train_features.pop('label')
train_features = np.array(train_features)


test_features = data_frame_test.copy()
test_labels = test_features.pop('label')
test_features = np.array(test_features)
        
x_train = train_features
y_train = train_labels
x_test = test_features[0:5000]
y_test = test_labels[0:5000]
x_val = test_features[5000:]
y_val = test_labels[5000:]  

In [8]:

concat_train_features = []
concat_test_features = []
for i in train_features:

    concat_train_features.append(i[0:1024] + i[1024:])

for i in test_features:

    concat_test_features.append(i[0:1024] + i[1024:])

x_train = concat_train_features
y_train = train_labels
x_test = concat_test_features[0:5000]
y_test = test_labels[0:5000]
x_val = concat_test_features[5000:10000]
y_val = np.array(test_labels[5000:10000])

print('X_train:',np.shape(x_train))
print('y_train:',np.shape(y_train))
print('X_test:',np.shape(x_test))
print('y_test:',np.shape(y_test))

X_train: (20000, 1024)
y_train: (20000,)
X_test: (5000, 1024)
y_test: (5000,)


In [38]:
basic_model = Sequential()
# Adding layers to the model
# First layers: 16 neurons/perceptrons that takes the input and uses 'sigmoid' activation function.
basic_model.add(Dense(units = 1024 , activation = 'relu', input_shape = (1024,))) 
basic_model.add(Dense(units = 512 , activation = 'relu')) 
basic_model.add(Dense(units = 256 , activation = 'relu')) 
basic_model.add(Dense(units = 64 , activation = 'relu')) 
basic_model.add(Dense(units = 32 , activation = 'relu')) 
basic_model.add(Dense(units = 16 , activation = 'relu')) 
basic_model.add(Dense(units = 8 , activation = 'relu')) 
basic_model.add(Dense(units = 1,activation = 'sigmoid')) 


In [39]:
optimizer = keras.optimizers.SGD(nesterov = True, lr=0.001, momentum=0.9)

In [40]:
basic_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [46]:
basic_model.fit(np.array(x_train), np.array(y_train), epochs=32)

Epoch 1/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1878 - accuracy: 0.9244
Epoch 2/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1811 - accuracy: 0.9286
Epoch 3/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1723 - accuracy: 0.9319
Epoch 4/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1619 - accuracy: 0.9360
Epoch 5/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1708 - accuracy: 0.9343
Epoch 6/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1582 - accuracy: 0.9379
Epoch 7/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1604 - accuracy: 0.9372
Epoch 8/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1537 - accuracy: 0.9393
Epoch 9/32
625/625 [==============================] - 3s 4ms/step - loss: 0.1499 - accuracy: 0.9414
Epoch 10/32
625/625 [==============================] - 3s 5ms/step - loss: 0.1439 - accuracy: 0.9457

In [ ]:
y_train.value_counts()

In [47]:
loss_and_metrics = basic_model.evaluate(np.array(x_test), np.array(y_test))
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

157/157 [==============================] - 1s 3ms/step - loss: 1.5610 - accuracy: 0.6436
Loss =  1.5609849691390991
Accuracy =  0.6435999870300293


In [48]:
loss_and_metrics

[1.5609849691390991, 0.6435999870300293]

In [49]:
basic_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_40 (Dense)            (None, 512)               524800    
                                                                 
 dense_41 (Dense)            (None, 256)               131328    
                                                                 
 dense_42 (Dense)            (None, 64)                16448     
                                                                 
 dense_43 (Dense)            (None, 32)                2080      
                                                                 
 dense_44 (Dense)            (None, 16)                528       
                                                                 
 dense_45 (Dense)            (None, 8)                

In [51]:
y_pred = basic_model.predict(np.array(x_test))

157/157 [==============================] - 0s 2ms/step


In [71]:
y_pred = np.round(y_pred)

In [64]:
len(y_pred[y_pred> 0.5])

1266

In [72]:
cm = confusion_matrix(y_test, y_pred)

In [74]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[1,0])
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)

NameError: name 'plt' is not defined